## Water Quality Data Analysis
### ENERES 131: Final Project
Team Members: Nidhi Nayak, Vishali Kirthi Vallioor, Damaris Arteaga, Timothy Zheng

![Water Quality](https://plus.unsplash.com/premium_photo-1710628263718-367b1cf5828f?q=80&w=2940&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D)

---

## Table of Contents and Deliverables
1. <a href="#abstract"><span style="color:red;">Section 1:</span> Abstract</a>
2. <a href="#project-background"><span style="color:red;">Section 2:</span> Project Background</a>
3. <a href="#project-objective"><span style="color:red;">Section 3:</span> Project Objective</a>
4. <a href="#input-data-description"><span style="color:red;">Section 4:</span> Input Data Description</a>
5. <a href="#data-cleaning"><span style="color:red;">Section 5:</span> Data Cleaning</a>
6. <a href="#eda"><span style="color:red;">Section 6:</span> Data Summary and Exploratory Data Analysis</a>
7. <a href="#forecasting-modeling"><span style="color:red;">Section 7:</span> Forecasting and Prediction Modeling </a>
   - <a href="#core-prediction"><span style="color:red;">Subsection 7.1:</span> Core Prediction Question</a>
   - <a href="#supporting-prediction"><span style="color:red;">Subsection 7.2:</span> Supporting Question</a>
   - <a href="#temporal-prediction"><span style="color:red;">Subsection 7.3:</span> Temporal Prediction Question</a>
8. <a href="#conclusions"><span style="color:red;">Section 8:</span> Interpretation and Conclusions</a>
9. <a href="#references"><span style="color:red;">Section 9:</span> Words Cited</a>

---

## <a id="abstract"></a>Section 1: Abstract

---

## <a id="project-background"></a>Section 2: Project Background

---

## <a id="project-objective"></a>Section 3: Project Objective


---

## <a id="input-data-description"></a>Section 4: Input Data Description

---

## <a id="data-cleaning"></a>Section 5: Data Cleaning

---

## <a id="eda"></a>Section 6: Data Summary and Exploratory Data Analysis

---

## <a id="forecasting-modeling"></a>Section 7: Forecasting and Prediction Modeling

---

### <a id="core-prediction"></a>Subsection 7.1: Core Prediction Question

---

### <a id="supporting-prediction"></a>Subsection 7.2: Supporting Question

---

### <a id="temporal-prediction"></a>Subsection 7.3: Temporal Prediction Question

---

## <a id="conclusions"></a>Section 8: Interpretation and Conclusions

---

## <a id="references"></a>Section 9: Words Cited

---